# Simulation template

In this notebook we run the machine learning analysis of topological phase transitions occurring  in both nearest-neighbours SSH models (ssh1) and second neighbours models (ssh2) as decribed in the paper [Machine learning topological phases in real space](https://arxiv.org/abs/1901.01963).

## Defining parameters

In [1]:
%%time
%load_ext autoreload
%autoreload 2
from simulation import *

CPU times: user 772 ms, sys: 245 ms, total: 1.02 s
Wall time: 572 ms


In [2]:
%%time
### Dataset and simulation parameters
csv_path = SSH1_PERIODIC_100_6561_CSV 
model_name = "DecisionTreeClassifier"
model_kw = {"criterion":"entropy"}
allowed_windings = [0,1]
simulation_dir = SSH1_PERIODIC_100_6561_SIMULATION_DIR
val_split = 0.9  
features_to_use = None 
shuffle_features = False
random_state = 137                    

### Running a simulation
n_experiments = 5
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
######### DON'T SET THIS TO TRUE UNLESS YOUR DATASET IS SMALL!! WILL FLOOD YOUR MEMORY!!!
store_in_lists = False   
########## BELOW ARE THE PARAMETERS THAT CONTROL WHAT WILL BE SAVED
save_eigenvector=True
save_hamiltonian=True 
save_accuracy=True 
save_models=True

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 5.72 µs


In [3]:
# Parameters
model_kw = {"criterion": "entropy", "n_estimators": 25, "n_jobs": -1}
allowed_windings = [-1, 0, 1, 2]
val_split = 0.5
features_to_use = None
shuffle_features = False
n_experiments = 5
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
store_in_lists = False
save_eigenvector = True
save_hamiltonian = True
save_accuracy = True
save_models = True
csv_path = "/home/rio/ssh_csvs/ssh2/periodic_140_6561.csv"
model_name = "RandomForestClassifier"
simulation_dir = "/home/rio/ssh_simulations/ssh2/periodic_140_6561"
random_state = 782


In [4]:
%%time
#Starting an instance of Simulation with a decision tree
model = DecisionTreeClassifier(criterion="entropy")
#dict_args = {"csv_path":csv_path, "model":model, "allowed_windings":allowed_windings,\
#             "simulation_dir":simulation_dir, "val_split":val_split, "features_to_use":features_to_use,\
#            "shuffle_features":shuffle_features, "random_state":random_state}
simulation = Simulation(csv_path,model_name,model_kw,allowed_windings,simulation_dir,val_split,features_to_use,\
                       shuffle_features,random_state)
#simulation = Simulation(**dict_args)

print("Info on all data: \n")
simulation.dataframe.info()
simulation.dataframe.head()

Info on all data: 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 918540 entries, 0 to 918539
Columns: 148 entries, id to feat139
dtypes: float64(143), int32(3), object(2)
memory usage: 1.0+ GB
CPU times: user 12.4 s, sys: 802 ms, total: 13.2 s
Wall time: 16 s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat130,feat131,feat132,feat133,feat134,feat135,feat136,feat137,feat138,feat139
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.130839,-0.300829,...,-0.080270,0.115548,-0.174799,-0.244861,0.319462,0.059061,-0.000568,0.244879,-0.300829,-0.130839
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.130839,-0.300829,...,0.080270,0.115548,0.174799,-0.244861,-0.319462,0.059061,0.000568,0.244879,0.300829,-0.130839
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.136032,0.040156,...,0.174723,0.086763,-0.070199,0.105493,-0.137161,-0.181862,0.180933,0.052357,-0.040156,0.136032
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.016772,0.038115,...,0.011447,-0.002690,0.015595,0.024714,-0.039364,-0.009687,0.001017,-0.028996,0.038115,0.016772
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.096087,0.024817,...,0.105770,0.025590,-0.024172,0.086413,-0.100604,-0.112297,0.113588,0.022113,-0.024817,0.096087


#### Checking initialization

In [5]:
%%time
n_features = simulation.n_features
n_hamiltonians = simulation.n_hamiltonians
n_ts = simulation.n_ts

print("n_features: ", n_features)
print("n_hamiltonians: ", n_hamiltonians)
print("n_ts: ", n_ts)

n_features:  140
n_hamiltonians:  6561
n_ts:  2
CPU times: user 186 µs, sys: 14 µs, total: 200 µs
Wall time: 118 µs


In [6]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.5450388660265203
% val:  0.0
% test:  0.45496113397347965
% train + val + test:  1.0


number of train hamiltonians:  3576
number of val hamiltonians:  0
number of test hamiltonians:  2985
total number of hamiltonians:  6561


train ids:  [41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315

## Running a simulation

In [7]:
%%time
simulation.run_simulation(n_experiments, start_n, fit_params, shuffle_rows, pred_params, random_features, \
                          store_in_lists, save_eigenvector, save_hamiltonian, save_accuracy,\
                          save_models)

simulation.dataframe.head(10)

running experiments:   0%|          | 0/5 [00:00<?, ?it/s]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:  20%|██        | 1/5 [00:36<02:25, 36.26s/it]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:  40%|████      | 2/5 [01:10<01:46, 35.62s/it]

...

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments: 100%|██████████| 5/5 [02:54<00:00, 35.31s/it]

running experiments: 100%|██████████| 5/5 [02:54<00:00, 34.95s/it]

CPU times: user 19min 2s, sys: 837 ms, total: 19min 2s
Wall time: 2min 54s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat130,feat131,feat132,feat133,feat134,feat135,feat136,feat137,feat138,feat139
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,-0.130839,-0.300829,...,-0.080270,0.115548,-0.174799,-0.244861,0.319462,0.059061,-0.000568,0.244879,-0.300829,-0.130839
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,0.130839,-0.300829,...,0.080270,0.115548,0.174799,-0.244861,-0.319462,0.059061,0.000568,0.244879,0.300829,-0.130839
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.136032,0.040156,...,0.174723,0.086763,-0.070199,0.105493,-0.137161,-0.181862,0.180933,0.052357,-0.040156,0.136032
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.016772,0.038115,...,0.011447,-0.002690,0.015595,0.024714,-0.039364,-0.009687,0.001017,-0.028996,0.038115,0.016772
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.096087,0.024817,...,0.105770,0.025590,-0.024172,0.086413,-0.100604,-0.112297,0.113588,0.022113,-0.024817,0.096087
5,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.037409,0.017407,...,0.064699,0.087166,-0.069474,-0.006214,-0.025629,-0.076251,0.078393,0.045301,-0.017407,0.037409
6,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.001081,0.008635,...,-0.052698,-0.054268,0.035200,-0.005245,0.001839,0.033479,-0.012524,-0.019661,0.008635,-0.001081
7,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,-0.037034,-0.082887,...,-0.029095,-0.011589,-0.023527,-0.045985,0.084698,0.025950,-0.004245,0.059527,-0.082887,-0.037034
8,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.105296,0.019814,...,0.076874,-0.034699,0.019704,0.110168,-0.114864,-0.090059,0.096749,-0.003643,-0.019814,0.105296
9,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.045950,0.023837,...,0.071514,0.118769,-0.103239,-0.027565,-0.023510,-0.100008,0.109383,0.069605,-0.023837,0.045950


#### Viewing a random sample

In [8]:
%%time
simulation.dataframe.sample(frac=0.1, replace=False)

CPU times: user 133 ms, sys: 0 ns, total: 133 ms
Wall time: 133 ms


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat130,feat131,feat132,feat133,feat134,feat135,feat136,feat137,feat138,feat139
260912,1863,H_-0.85_-2_2.00552.dat,-0.85,-2.00,2.005517,999,1,test,0.117982,-0.006159,...,0.060309,0.061884,0.043428,-0.136665,0.068104,0.047480,0.070715,-0.039249,-0.006159,0.117982
903636,6454,H_1.95_0.75_-1.00221.dat,1.95,0.75,-1.002214,999,0,test,0.010487,0.097363,...,0.068243,-0.100201,-0.078119,0.104475,0.110713,0.019803,-0.024837,-0.091086,-0.097363,-0.010487
686077,4900,H_1_0_0.dat,1.00,0.00,0.000000,999,1,test,-0.007143,-0.007143,...,0.058202,0.058202,-0.048609,-0.048609,0.038626,0.038626,-0.028331,-0.028331,0.017809,0.017809
321536,2296,H_-0.6_-0.6_0.501338.dat,-0.60,-0.60,0.501338,999,0,test,0.074050,-0.073093,...,-0.119477,0.120018,-0.061294,0.096729,0.017359,0.014947,0.078950,-0.067980,0.073093,-0.074050
441463,3153,H_-0.1_1.75_2.00141.dat,-0.10,1.75,2.001407,999,0,test,0.061697,0.085655,...,-0.102662,0.014529,-0.117988,-0.103608,0.041252,-0.097707,0.010912,-0.061807,0.085655,0.061697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220889,1577,H_-1.05_-0.1_-0.00256881.dat,-1.05,-0.10,-0.002569,0,0,train,-0.051996,0.110423,...,0.032430,0.084922,0.108033,-0.038517,0.067008,-0.105143,-0.046654,-0.059764,-0.110423,0.051996
843559,6025,H_1.7_-0.45_-0.999899.dat,1.70,-0.45,-0.999899,999,1,test,0.104157,0.003419,...,-0.106303,0.023396,0.003915,-0.109753,0.102075,0.083756,-0.102441,0.032725,-0.003419,-0.104157
198404,1417,H_-1.15_0_-0.00202773.dat,-1.15,0.00,-0.002028,0,0,train,0.119522,-0.050520,...,0.107909,0.001482,0.074118,-0.116188,-0.074924,-0.053190,-0.107462,0.092516,0.027099,0.094363
338888,2420,H_-0.55_1.55_1.0008.dat,-0.55,1.55,1.000801,999,0,test,0.045120,-0.070979,...,-0.104528,-0.044934,-0.121988,0.096762,0.024457,0.105047,0.035891,0.063103,0.070979,-0.045120


#### Checking train/val/test splits again

In [9]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.27251943301326015
% val:  0.27251943301326015
% test:  0.45496113397347965
% train + val + test:  1.0


number of train hamiltonians:  1788
number of val hamiltonians:  1788
number of test hamiltonians:  2985
total number of hamiltonians:  6561


train ids:  [4455, 3210, 3089, 1444, 68, 1691, 4812, 1006, 3191, 795, 1010, 1488, 2413, 2732, 2445, 2698, 2016, 1101, 3985, 1337, 1347, 4248, 1761, 77, 3979, 218, 394, 1897, 5428, 1404, 4711, 2145, 1287, 850, 5998, 3593, 1569, 2061, 2658, 627, 4472, 6331, 4621, 964, 4137, 1441, 2534, 5429, 5593, 3821, 1839, 4704, 2893, 3163, 2403, 3136, 765, 864, 2762, 3330, 292, 1749, 4074, 4626, 5440, 309, 319, 706, 5450, 1174, 5353, 4070, 2887, 1746, 775, 2331, 5279, 547, 3055, 1759, 2895, 1932, 3678, 5687, 4714, 1040, 4565, 3646, 5288, 6242, 3102, 1191, 714, 2878, 5767, 2221, 3829, 3339, 3357, 1345, 3208, 2224, 4417, 2638, 2080, 5695, 5052, 4214, 1996, 3929, 457, 1923, 3091, 2636, 3047, 6339, 4089, 4896, 1178, 217, 2251, 768, 3253, 3748, 801, 4

#### Checking summaries

In [10]:
%%time
ham_summary = simulation.hamiltonian_summary
print("length of ham_summary: ", len(ham_summary))
ham_summary

length of ham_summary:  6561
CPU times: user 367 µs, sys: 0 ns, total: 367 µs
Wall time: 259 µs


,t1,t2,type_of,0,1,phase,pred_phase
id,,,,,,,
0,-2.0,-2.00,test,0.471429,0.528571,999,0
1,-2.0,-1.95,test,0.492857,0.507143,999,0
2,-2.0,-1.90,test,0.550000,0.450000,999,-1
3,-2.0,-1.85,test,0.578571,0.421429,999,-1
4,-2.0,-1.80,test,0.557143,0.442857,999,-1
...,...,...,...,...,...,...,...
6556,2.0,1.80,test,0.678571,0.321429,999,-1
6557,2.0,1.85,test,0.707143,0.292857,999,-1
6558,2.0,1.90,test,0.707143,0.292857,999,-1


In [11]:
eigen_summary = simulation.eigenvector_summary
print("length of ham_summary: ", len(eigen_summary))
eigen_summary

length of ham_summary:  918540


,id,phase,pred_phase,type_of
0,0,999,0,test
1,0,999,0,test
2,0,999,1,test
3,0,999,1,test
4,0,999,1,test
...,...,...,...,...
918535,6560,999,0,test
918536,6560,999,0,test
918537,6560,999,0,test
918538,6560,999,0,test


#### Checking accuracies

In [12]:
simulation.accuracy

{'eigenvector_train': 0.9998841482901886,
 'eigenvector_val': 0.9888023330137424,
 'eigenvector_test': 0.8067927170868348,
 'hamiltonian_train': 0.0,
 'hamiltonian_val': 0.0,
 'hamiltonian_test': 0.00980392156862745}

#### Checking data stored in  memory

In [13]:
ham_summary_list = simulation.hamiltonian_summary_list
ham_summary_list

[]

In [14]:
eigen_summary_list = simulation.eigenvector_summary_list
eigen_summary_list

[]

In [15]:
accuracy_list = simulation.accuracy_list
accuracy_list

{'eigenvector_train': [],
 'eigenvector_val': [],
 'eigenvector_test': [],
 'hamiltonian_train': [],
 'hamiltonian_val': [],
 'hamiltonian_test': []}